# Сравнение средних

**Содержание:**
* [Введение](#Введение)  
* [Центральная предельная теорема](#Центральная-предельная-теорема)
* [Оценка среднего значения распределения](#Оценка-среднего-значения-распределения)
* [Сравнение средних двух распределений](#Сравнение-средних-двух-распределений)
* [Приложение: сопряженное априорное распределение к нормальному распределению](#Приложение:-сопряженное-априорное-распределение-к-нормальному-распределению)
* [Приложение: оценка параметров нормального распределения по сэмплу](#Приложение:-оценка-параметров-нормального-распределения-по-сэмплу)
* [Заключение](#Заключение)  

## Введение

В веб-сервисы и мобильные приложения с течением времени вносятся изменения. Оценивать эффект от изменений можно сравнением метрик. При этом сравнение лучше проводить в рамках А/Б-тестов, чтобы минимизировать различие во внешних факторах.  

Наибольший интерес представляют метрики, заложенные в план развития продукта. Сравнение конверсий было рассмотрено в предыдущей части. Могут быть интересны денежные метрики и метрики вовлеченности - выручка на пользователя, время в приложении, длительность просмотра видео и т.д.  

<картинка>  

Байесовский подход требует построения моделей распределений сравниваемых величин. Выбор модели - неочевидный вопрос. Есть отдельные модели, получившие распространение - например, [Buy Till You Die](https://en.wikipedia.org/wiki/Buy_Till_you_Die) для моделирования количества покупок клиента за все время использования сервиса. Но универсальных моделей нет. Это создает сложности.

С другой стороны, для многих величин не всегда нужно знать все распределение. 
Можно ограничиться сравнением средних:
средней выручкой на пользователя, средней длительностью просмотра и т.д.

Для средних значений часто применима центральная предельная теорема. Можно приближенно считать, что средние значения в выборках из распределения будут распределены нормально. Причем это не зависит от формы исходного распределения.  

Центральная предельная теорема позволяет при сравнении средних использовать нормальное распределение в качестве функции правдоподобия.

В ч.1 обсуждался байесовским подход к оценке А/Б-тестов.  
На примере конверсий было показано, как в рамках этого подхода ответить на вопросы
- Какой вариант лучше и насколько?
- Каковы оценки целевой метрики в каждом варианте?
- Насколько уверены в оценке?
- Сколько должен продолжаться эксперимент?

В этой части будет рассмотрен вопрос о том, как ответить на эти же вопросы применительно к сравнению средних.

Структура следующая:
- вначале обсуждается центральная предельная теорема
- далее проводится оценка средних для распределений с известными параметрами
- проводится сравнение средних для распределений с разными параметрами и делается попытка ответить на вопросы выше

## Центральная предельная теорема

Есть несколько центральных предельных теорем [[CLT](https://en.wikipedia.org/wiki/Central_limit_theorem)].
Одна из возможных формулировок следующая. Пусть есть последовательность независимых одинаково распределенных случайных величин $X_1, X_2, \dots, X_n, \dots$ с конечным математическим ожиданием $\mu$ и дисперсией $\sigma^2$. Пусть $\bar{X}_n = \frac{1}{n} \sum_{i=1}^{n} X_i$ их выборочное среднее. Тогда при $n$, стремящемся к бесконечности, распределение центрированных и масштабированных выборочных средних сходится к нормальному распределению со средним значением 0 и дисперсией 1

$$
\frac{\bar{X}_n - \mu}{(\sigma / \sqrt{n})} \to N(0, 1), \quad n \to \infty.
$$

Сходимость понимается как сходимость по распределению [[RandVarsConvergences](https://en.wikipedia.org/wiki/Convergence_of_random_variables#Convergence_in_distribution)].

Неформально этот результат можно применить следующим образом. Если взять произвольное распределение со средним значением $\mu$ и диспресий $\sigma^2$, начать выбирать из него сэмплы длины $n$ и считать среднее в каждом сэмпле, то средние значения сэмплов будут распределены приблизительно нормально $N(\mu, \sigma^2/n)$.

<center>
<img src="central_limit_theorem.png" alt="central_limit_theorem" width="600"/>
</center>

Приведенная формулировка центральной предельной теоремы говорит о сходимости к нормальному распределению центрированных и масштабированных выборочных средних $\bar{X}_n$ при стремлении $n$ к бесконечности. Для фиксированного конечного числа $n$ нормальное распределение не гарантируется. При этом есть теоремы, дающие оценку отличия распределения суммы конечного количества случайных величин от нормального - см. [[BerryEssenTheorem](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem)]. Отличие зависит как от количества слагаемых, так и от параметров распределения. 

<На практике бывает достаточно нескольких десятков точек в каждом сэмпле.>  

In [ ]:
import pandas as pd
import numpy as np
np.random.seed(7)

from collections import namedtuple

import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#todo: update scipy; make venv

In [ ]:
a = 1
sample_len = 100
n_samples = 1000

exact_dist = stats.gamma(a=a)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)

x = np.linspace(0, 10, 10000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', name='Exact Dist'))
fig.add_trace(go.Histogram(x=samp[0], histnorm='probability density', name='Sample'))
fig.update_layout(title=f'Original Distribution and a Sample of Length {sample_len}',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Means of Samples'))
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=clt_mu, scale=clt_stdev), 
                         mode='lines', name='CLT Means Distrib'))
fig.update_layout(title='Sample Means Distribution',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

В приведенной формулировке центральная предельная теорема требует существования конечных среднего и дисперсии у исходного распределения. Примерами распределений, для которых эти свойства могут не выполнятся, являются распределение Парето [[ParetoDist](https://en.wikipedia.org/wiki/Pareto_distribution), [SciPyPareto](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pareto.html)] и близкое к нему распределение Ломакса [[LomaxDist](https://en.wikipedia.org/wiki/Lomax_distribution), [SciPyLomax](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.lomax.html)]. Плотность вероятности последнего имеет вид

$$
P(x; c) = \frac{c}{(1 + x )^{c + 1}}, \quad x \ge 0, c > 0.
$$

При значениях параметра $c$ меньше или равном 2 дисперсия распределения Ломакса не является конечной.  

На графиках ниже приведена гистограмма средних в сэмплах и нормальное распределение с параметрами, равными среднему и дисперсии средних сэмплов.

In [ ]:
c = 1.7
sample_len = 500
n_samples = 1000

exact_dist = stats.lomax(c=c)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)
means_stdev = means.std()

x = np.linspace(0, 30, 10000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', name='Exact Dist'))
fig.add_trace(go.Histogram(x=samp[0], histnorm='probability density', name='Sample'))
fig.update_layout(title=f'Original Distribution and a Sample of Length {sample_len}',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Means of Samples'))
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=clt_mu, scale=means_stdev), 
                         mode='lines', name='CLT-like Normal'))
fig.update_layout(title='Sample Means Distribution',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

Видно, что распределение средних сильнее отличается от нормального, чем в предыдущем примере - оно скошено в сторону больших значений. Это объясняется тем, что плотность вероятности в распределениях Паретто и Ломакса "медленно" убывает с ростом $x$ ("тяжелый хвост"). Если в сэмпл попадает значение из хвоста, среднее сэмпла смещается. В итоге распределение средних перестает быть нормальным и становится скошенным в сторону больших значений.  

<Есть обобщенная центральная предельная теорема https://en.wikipedia.org/wiki/Stable_distribution#A_generalized_central_limit_theorem, которая говорит 
про предельное распределение средних в этом случае>.  

# Оценка среднего значения распределения

Пусть задано распределение и есть сэмпл из него.  
Нужно оценить среднее значение распределения.  

In [ ]:
a = 1
exact_dist = stats.gamma(a=a)

sample_size = 1000
samp = exact_dist.rvs(size=sample_size)
exact_mean = exact_dist.mean()
sample_mean = samp.mean()

x = np.linspace(0, 10, 10000)
ymax = np.max(exact_dist.pdf(x))
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', name='Exact'))
fig.add_trace(go.Histogram(x=samp, histnorm='probability density', name='Sample'))
fig.add_trace(go.Scatter(x=[exact_mean, exact_mean], y=[0, ymax], 
                         mode='lines', line_color='black', 
                         name='Exact Mean'))
fig.add_trace(go.Scatter(x=[sample_mean, sample_mean], y=[0, ymax], 
                         mode='lines', line_dash='dash', line_color='black', 
                         name='Sample Mean'))
fig.update_layout(title='Dist',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()

Точечную оценку среднего распределения дает среднее значение в сэмпле.  
Но кроме точечных оценок нужны распределения возможных значений средних.   
Для этого можно использовать центральную предельную теорему.

По центральной предельной теореме можно ожидать, что средние значения сэмплов будут распределены нормально.  
Поэтому можно выбрать нормальное распределение в качестве функции правдоподобия и с помощью соотношения Байеса построить распределение вероятности возможных параметров.  

Можно посчитать среднее по всем точкам сэмпла после чего оценивать распределения параметров модели для получения этого значения. Проблема в том, что по одной точке нельзя понять, выполняется ли центральная предельная теорема и можно ли применять нормальную функцию правдоподобия.

Более устойчивым способом кажется разбить сэмпл на части по $n$ точек (например, $n = 25$). В каждой части посчитать среднее. После чего оценить распределение параметров при приближении распределения средних нормальным распределением. В таком подходе появляется возможность валидировать модель.

<картинка>

Полезно построить график средних в сэмплах и визуально сравнить его с нормальным. Вид гистограммы средних будет меняться в зависимости от выбранного n_split. Можно попробовать подбирать n_split как гиперпараметр и выбирать значение по итогам кросс-валидации. Пока выбрано фиксированное значение n_split = 25. *Можно попробовать оценить n_split из неравенства Берри-Эссена [[BerryEssenTheorem](https://en.wikipedia.org/wiki/Berry%E2%80%93Esseen_theorem)]*. 

Т.к. исходное распределение неизвестно, для параметров нормального распределения средних можно использовать среднее и дисперсию из выборки. 

In [ ]:
def reshape_and_compute_means(sample, n_split):
    n_means = len(sample) // n_split
    samp_reshaped = np.reshape(sample[0 : n_means * n_split], (n_means, n_split))
    means = np.array([x.mean() for x in samp_reshaped])
    return means

def exact_clt_dist(exact_dist, n_split):
    clt_mu = exact_dist.mean()
    clt_stdev = exact_dist.std() / np.sqrt(n_split)
    return stats.norm(loc=clt_mu, scale=clt_stdev)

def sample_clt_dist(means):
    clt_mu = means.mean()
    clt_std = means.std()
    return stats.norm(loc=clt_mu, scale=clt_std)

In [ ]:
n_split = 25
means = reshape_and_compute_means(samp, n_split)

x = np.linspace(0, 10, 10000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Means of Samples'))
fig.add_trace(go.Scatter(x=x, y=sample_clt_dist(means).pdf(x), mode='lines', name='CLT-like Normal'))
fig.update_layout(title='Means of Samples and CLT-like Normal',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

*Хочется иметь численный критерий, что можно использовать нормальное распределение.*  
*По-хорошему, кросс-валидация + нужно решить, что делать с ее результатами.*  
*Посмотреть, есть ли что-то в https://en.wikipedia.org/wiki/Normality_test*.     

В байесовском подходе для оценки плотности вероятности параметров модели используется соотношение

$$
P(model | data) = \frac{ P(data | model) P(model) }{P(data)}.
$$

Для построения модели распределения средних байесовским методом функцию правдоподобия можно задать в виде нормального распределения:

$$
P(data | model) = N(x; \mu, \frac{\sigma_{0}^2}{\lambda} ) ,  
\\
N(x ; \mu, \frac{\sigma_{0}^2}{\lambda}) = \frac{\lambda^{1/2}}{\sqrt{2 \pi \sigma_{0}^2}} e^{- \frac{\lambda}{2 \sigma_{0}^2} (x-\mu)^2} .
$$

Можно показать (см. приложение), что для такой функции правдоподобия произведение нормального и гамма-распределений будет сопряженным априорным распределением:
$$
P(model | data) = N(\mu; \mu_i, \frac{\sigma_0^2}{k_i \lambda} ) Gamma(\lambda; a_i, b_i) ,  
\\
Gamma(\lambda; a,b) = \frac{b^a}{\Gamma(a)} \lambda^{a-1} e^{-b \lambda} .
$$

Параметры интерпретируются следующим образом:   

<картинка и описание>  

Параметры $\mu_i \to \mu_{CLT}$ и $\sigma_{0}^2 / \lambda \to \sigma^2_{CLT}$ (?определить $\sigma_{CLT}$; верно ли $\sigma_{0}^2 / \lambda  \to \sigma^2 / n$?).  
$\lambda$ - отношение дисперсии модели к дисперсии ЦПТ,
$\sigma_0^2$ задается по историческим данным, $\lambda$ варьируется.  
Параметры $a_i, b_i$ определяют точное распределение этого коэффициента.   


Обновление параметров сопряженного распределения с каждой новой точкой $x_i$ выполняется
по соотношениям:

$$
k_{i+1} = k_i + 1 ,  
\\
\mu_{i+1} = \frac{x_i + k_i \mu_i}{k_i + 1} ,  
\\
a_{i+1} = a_i + 1/2 , 
\\
b_{i+1} = b_i + \frac{k_i}{k_i + 1} \frac{(x_i - \mu_i)^2}{2 \sigma_0^2} . 
$$

Исходные значения параметров $\mu_0$ и $\sigma_0$ можно задать произвольно. Но если они будут далеки
от реальных, то модель может оказаться неточной. Поэтому удобнее задавать их либо
по историческим данным, либо на основе части данных из сэмпла. Для $\mu_0$ можно
выбрать среднее $\bar{x}_1$ в первом наборе из $n$ точек, для $\sigma_0^2$ - дисперсию в этой
части сэмпла $\sigma_{\bar{x}_1}^2$, отнесенную к количеству точек $n_{split}$.
Для начальных значений $a_0$ и $b_0$ можно выбрать 2 и 1, поскольку при таких значениях
параметров мода $p(\lambda) = Gamma(\lambda, a=2, b=1)$ будет равна 1. Значение
$k_0$ можно выбрать $1/25$, чтобы вначале обеспечить широкую область для $P(\mu) = N(\mu_0, (5 \sigma_0)^2)$.

$$
\mu_0 = \bar{x}_1, \sigma_0 = \sigma_{\bar{x}_1} / \sqrt{n_{split}} ,
\\ 
k_0 = 1/25 ,
\\
a_0 = 2, b_0 = 1 .
$$

In [ ]:
ConjugateNormalParams = namedtuple('ConjugateNormalParams', 'mu sigma k a b')

def initial_parameters(mu, sigma, k=1/25, a=2, b=1):
    return ConjugateNormalParams(mu=mu, sigma=sigma, k=k, a=a, b=b)

def update_conj_parameters(x, conj_norm_pars):
    mu_p = (x + conj_norm_pars.k * conj_norm_pars.mu) / (conj_norm_pars.k + 1)
    sigma_p = conj_norm_pars.sigma
    k_p = conj_norm_pars.k + 1
    a_p = conj_norm_pars.a + 1/2
    b_p = conj_norm_pars.b + conj_norm_pars.k / (conj_norm_pars.k + 1) * (x - conj_norm_pars.mu)**2 / (2 * conj_norm_pars.sigma**2)
    return ConjugateNormalParams(mu=mu_p, sigma=sigma_p, k=k_p, a=a_p, b=b_p)

def compute_posterior_parameters(sample, n_split):
    means = reshape_and_compute_means(sample, n_split)
    mean_1 = means[0]
    sigma_1 = sample[0:n_split].std() / np.sqrt(n_split)
    pars = []
    pars.append(initial_parameters(mu=mean_1, sigma=sigma_1)) 
    for x in means[1:]:
        new_pars = update_conj_parameters(x, pars[-1])
        pars.append(new_pars)
    return pars

#pars_initial = initial_parameters(mu=1, sigma=2)
#print(pars_initial)
#print(update_conj_parameters(1.3, pars_initial))
#pars = compute_posterior_parameters(samp, n_splilt)
#print(pars[-1])

Для анализа параметров и сравнения групп вместо двумерного распределения параметров $P(\mu, \lambda | data)$ удобнее сравнивать одномерные маржинальные распределения по соответствующим параметрам $P(\lambda | data), P(\mu | data)$.

Маржинальное апостериорное распределение $P(\lambda | data)$ по конструкции задается гамма-распределением:

$$
P(\lambda | data) = Gamma(\lambda; a_i, b_i).
$$

Для интерпретации удобнее величина $\sigma = \sigma_0 / \lambda^{1/2}$, распределение которой можно получить с помощью [замены переменных](https://en.wikipedia.org/wiki/Probability_density_function#Function_of_random_variables_and_change_of_variables_in_the_probability_density_function) :

$$
P(\sigma | data) = Gamma\left( \frac{\sigma_0^2}{\sigma^2 }; a_i, b_i \right) \frac{2 \sigma_0^2}{\sigma^3}, 
\quad \sigma > 0 .
$$

Можно показать (см. приложение), что маржинальное апостериорное распределение $P(\mu | data)$ задается $t$-распределением:

$$
P(\mu | data) = t(\mu | \nu = 2a_i, \mu_t = \mu_i, \sigma_t^2 = \frac{\sigma_0^2}{k_i} \frac{b_i}{a_i} ).
$$

Для апостериорного распределения нужно вначале сгенерировать параметры модели, потом подставить их в функцию правдоподобия. Т.е. нужно произведение соответствующих распределений:

$$
P(x) = N(x; \mu, \frac{\sigma_0^2}{\lambda} ) N(\mu; \mu_i, \frac{\sigma_0^2}{k_i \lambda} ) Gamma(\lambda; a_i, b_i) .
$$

In [ ]:
def mu_marginal_distrib(conj_norm_pars):
    #ok; function results coincide with manual comp
    df = 2 * conj_norm_pars.a
    loc = conj_norm_pars.mu
    scale = np.sqrt(conj_norm_pars.sigma**2 / conj_norm_pars.k * conj_norm_pars.b / conj_norm_pars.a)
    return stats.t(df=df, loc=loc, scale=scale)

def lambda_marginal_distrib(conj_norm_pars):
    return stats.gamma(a=conj_norm_pars.a, scale=1/conj_norm_pars.b)

def sigma_marginal_distrib_pdf(x, conj_norm_pars):
    #ok; function results coincide with hist of sigma_0/sqrt(lambda)
    dl_ds = 2 * conj_norm_pars.sigma**2 / np.abs(x**3)
    return lambda_marginal_distrib(conj_norm_pars).pdf(conj_norm_pars.sigma**2 / x**2) * dl_ds

PosteriorParams = namedtuple('PosteriorParams', 'mu lmd sigma')

def post_params_rvs(conj_norm_pars, size):
    lmds = stats.gamma.rvs(a=conj_norm_pars.a, scale=1/conj_norm_pars.b, size=size)
    mus = stats.norm.rvs(loc=conj_norm_pars.mu, scale=conj_norm_pars.sigma / np.sqrt(lmds * conj_norm_pars.k))
    #return [{'mu': mu, 'lambda': l, 'sigma':conj_norm_pars.sigma} for mu, l in zip(mus, lmds)]
    return [PosteriorParams(mu, l, conj_norm_pars.sigma) for mu, l in zip(mus, lmds)]
    
def post_means_rvs(conj_norm_pars, size):
    #ok; function results coincide with manual comp
    lmd = lambda_marginal_distrib(conj_norm_pars).rvs(size=size)
    post_mu = stats.norm.rvs(loc=conj_norm_pars.mu, scale=conj_norm_pars.sigma / np.sqrt(lmd * conj_norm_pars.k))
    post_means = stats.norm.rvs(loc=post_mu, scale=conj_norm_pars.sigma / np.sqrt(lmd))
    return post_means

Распределения параметров $\mu$ и $\sigma$ приведены ниже:

In [ ]:
pars = compute_posterior_parameters(samp, n_split)

#mu
mu_dist = mu_marginal_distrib(pars[-1])
x = np.linspace(0, 10, 10000)
yplot = mu_dist.pdf(x)
ymax = max(yplot)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=mu_dist.pdf(x), mode='lines', name=f'Mu Estimate Distrib'))
fig.add_trace(go.Scatter(x=[means.mean(), means.mean()], y=[0, ymax],
                         mode='lines', line_color='black', line_dash='dash', 
                         name='Sample Mean'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, ymax], 
                         mode='lines', line_color='black',
                         name='Exact Mean'))
fig.update_layout(title='Dist',
                  xaxis_title='mu',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

# sigma
fig = go.Figure()
x = np.linspace(0.001, 10, 10000)
x_plot = x
y_plot = sigma_marginal_distrib_pdf(x, pars[-1])
y_max = max(y_plot)
fig.add_trace(go.Scatter(x=x, 
                         y=y_plot, 
                         mode='lines', line_dash='dash', name=f'Sigma Estimate Distrib'))
# lambdas = lambda_marginal_distrib(pars[-1]).rvs(size=50000)
# sigmas = pars[-1].sigma / np.sqrt(lambdas)
# fig.add_trace(go.Histogram(x=sigmas, histnorm='probability density', name='Sigma Hist',
#                            opacity=0.7))
fig.add_trace(go.Scatter(x=[means.std(), means.std()], 
                         y=[0, y_max],
                         mode='lines', line_color='black', line_dash='dash', 
                         name='Sample Stdev'))
fig.add_trace(go.Scatter(x=[exact_clt_dist(exact_dist, n_split).std(), exact_clt_dist(exact_dist, n_split).std()], 
                         y=[0, y_max], 
                         mode='lines', line_color='black',
                         name='Exact Stdev'))
fig.update_layout(title='Dist',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  barmode="overlay",
                  height=550)
fig.update_layout(xaxis_range=[0, 1])
fig.show()

Видно, что моды распределений близки к значениям среднего и дисперсии в сэмпле. Также точные значения имеют "большое" значение плотности вероятности.

Распределение сгенерированных апостериорных средних:

In [ ]:
#post means
post_means = post_means_rvs(pars[-1], size=50000)

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.add_vline(means.mean(), line_dash='dash', name='Sample Mean')
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Means of Samples',
                           opacity=0.7, marker_color='green'))
#fig.add_trace(go.Scatter(x=x, y=exact_clt_dist(exact_dist, n_split).pdf(x), mode='lines', name='CLT Distribution'))
fig.add_trace(go.Scatter(x=x, y=sample_clt_dist(means).pdf(x), mode='lines', name='CLT-like Distribution'))
fig.add_trace(go.Scatter(x=x, y=mu_dist.pdf(x), mode='lines', name=f'Mu Estimate Distrib'))    
   
fig.add_trace(go.Histogram(x=post_means, histnorm='probability density', name='Posterior Means', opacity=0.7))
fig.update_layout(title='Dist',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.update_layout(xaxis_range=[0, 5])
fig.show()

Распределение сгенерированных апостериорных средних обычно несколько шире, чем распределение центральной предельной теоремы. Это объясняется дисперсией в параметрах $\mu$ и $\lambda$ (в ЦПТ $\mu$ и $\sigma$ фиксированные числа). 

Для наглядности удобно нарисовать распределение параметра $\mu_i$ и распределение средних на одном графике.
Распределение средних шире, чем распределение параметра $\mu$. Это связано с учетом дисперсии $\sigma_0/\lambda^{1/2}$. 

# Сравнение средних двух распределений

Пусть есть два распределения одинаковой формы, но с разными средними значениями.
Для примера

$$
x \sim Gamma(a, b),
\quad
\bar{x} = \frac{a}{b},
\quad
\sigma^2 = \frac{a}{b^2}
.
$$

В одном случае параметры $a_A = 1, b_A = 1$ и среднее $\bar{x}_A = 1$, в другом параметры $a_B = 1, b_B = 1.25$ и среднее $\bar{x}_B = 0.8$. При выбранных значениях параметров среднее в группе $B$ отличается на $20\%$ от группы $A$ (относительно $A$).

In [ ]:
exp_info = pd.DataFrame([
    {'group':'A', 'a':1, 'b':1, 'sample_size':10000},
    {'group':'B', 'a':1, 'b':1.25, 'sample_size':10000}])
exp_info.set_index('group', inplace=True)

exact_dist_a = stats.gamma(a=exp_info['a']['A'], scale=1/exp_info['b']['A'])
exact_dist_b = stats.gamma(a=exp_info['a']['B'], scale=1/exp_info['b']['B'])
exp_info['exact_mean'] = pd.Series({'A': exact_dist_a.mean(), 'B':exact_dist_b.mean()})
display(exp_info)

samp_a = exact_dist_a.rvs(size=exp_info['sample_size']['A'])
samp_b = exact_dist_b.rvs(size=exp_info['sample_size']['B'])

x = np.linspace(0, 10, 10000)
ymax = np.max([exact_dist_a.pdf(x), exact_dist_b.pdf(x)])

fig = go.Figure()
col = 'red'
fig.add_trace(go.Scatter(x=x, y=exact_dist_a.pdf(x), 
                         mode='lines', line_color=col,
                         name=f"Exact A: a={exp_info['a']['A']}, b={exp_info['b']['A']}"))
fig.add_trace(go.Histogram(x=samp_a, histnorm='probability density',
                           name='Sample A',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=[exact_dist_a.mean(), exact_dist_a.mean()], y=[0, ymax], 
                         mode='lines', line_color=col, line_dash='dash',
                         name='Exact Mean A'))
col = 'blue'
fig.add_trace(go.Scatter(x=x, y=exact_dist_b.pdf(x), 
                         mode='lines', line_color=col,
                         name=f"Exact B: a={exp_info['a']['B']}, b={exp_info['b']['B']}"))
fig.add_trace(go.Histogram(x=samp_b, histnorm='probability density',
                           name='Sample B',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=[exact_dist_b.mean(), exact_dist_b.mean()], y=[0, ymax], 
                         mode='lines', line_color=col, line_dash='dash',
                         name='Exact Mean B'))
fig.update_layout(title='Exact Distributions and Samples',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode='overlay')
fig.show()

Распределение средних на основе точек из сэмплов:

In [ ]:
n_split = 25

means_a = reshape_and_compute_means(samp_a, n_split)
means_b = reshape_and_compute_means(samp_b, n_split)

x = np.linspace(0, 10, 10000)
ymax = np.max([exact_dist_a.pdf(x), exact_dist_b.pdf(x), 
               sample_clt_dist(means_a).pdf(x), sample_clt_dist(means_b).pdf(x)])


fig = go.Figure()
col = 'red'
fig.add_trace(go.Histogram(x=means_a, histnorm='probability density',
                           name='A Sample Means',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=x, y=sample_clt_dist(means_a).pdf(x), 
                         mode='lines', line_color=col, opacity=0.5,
                         name=f"A Sample CLT"))
fig.add_trace(go.Scatter(x=x, y=exact_dist_a.pdf(x), 
                         mode='lines', line_color=col, line_dash='dash', opacity=0.3,
                         name=f"A Exact: a={exp_info['a']['A']}, b={exp_info['b']['A']}"))
# fig.add_trace(go.Scatter(x=[means_a.mean(), means_a.mean()], y=[0, ymax],
#                          mode='lines', line_color=col, line_dash='dash', 
#                          name='A Sample Mean'))
col = 'blue'
fig.add_trace(go.Histogram(x=means_b, histnorm='probability density',
                           name='B Sample Means',
                           opacity=0.3, marker_color=col))
fig.add_trace(go.Scatter(x=x, y=sample_clt_dist(means_b).pdf(x), 
                         mode='lines', line_color=col, opacity=0.5,
                         name=f"B Sample CLT"))
fig.add_trace(go.Scatter(x=x, y=exact_dist_b.pdf(x), 
                         mode='lines', line_color=col, line_dash='dash', opacity=0.3,
                         name=f"B Exact: a={exp_info['a']['B']}, b={exp_info['b']['B']}"))
# fig.add_trace(go.Scatter(x=[means_b.mean(), means_b.mean()], y=[0, ymax],
#                          mode='lines', line_color=col, line_dash='dash', 
#                          name='B Sample Mean'))
fig.update_layout(title='Sample Means',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  barmode='overlay',
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()
#
exp_info['exact_mean'] = pd.Series({'A': exact_dist_a.mean(), 'B': exact_dist_b.mean()})
exp_info['sample_mean'] = pd.Series({'A': samp_a.mean(), 'B': samp_b.mean()})
exp_info

Расчет параметров с учетом данных:

In [ ]:
pars_a = compute_posterior_parameters(samp_a, n_split)
pars_b = compute_posterior_parameters(samp_b, n_split)

print(pars_a[-1])
print(pars_b[-1])

Распределения параметров $\mu$, $\sigma$ и апостериорных средних:

In [ ]:
#mu
mu_dist_a = mu_marginal_distrib(pars_a[-1])
mu_dist_b = mu_marginal_distrib(pars_b[-1])
x_mu = np.linspace(0, 10, 10000)
ymax_mu = np.max([mu_dist_a.pdf(x_mu), mu_dist_b.pdf(x_mu)]) #todo: optimize

#sigam
x_sg = np.linspace(0.01, 10, 10000)
sigma_dist_a = sigma_marginal_distrib_pdf(x_sg, pars_a[-1])
sigma_dist_b = sigma_marginal_distrib_pdf(x_sg, pars_b[-1])
ymax_sg = np.max([sigma_dist_a, sigma_dist_b]) #todo: optimize

#posterior
post_means_a = post_means_rvs(pars_a[-1], size=30000)
post_means_b = post_means_rvs(pars_b[-1], size=30000)
x_post = np.linspace(0, 10, 10000)


fig = go.Figure()
col = 'red'
fig.add_trace(go.Scatter(x=x_mu, y=mu_dist_a.pdf(x), 
                         mode='lines', line_color=col, 
                         name=f'A Mu Dist'))
fig.add_trace(go.Scatter(x=[means_a.mean(), means_a.mean()], y=[0, ymax_mu],
                         mode='lines', line_color=col, line_dash='dash', 
                         name='A Sample Mean'))
col = 'blue'
fig.add_trace(go.Scatter(x=x_mu, y=mu_dist_b.pdf(x), 
                         mode='lines', line_color=col, 
                         name=f'B Mu Dist'))
fig.add_trace(go.Scatter(x=[means_b.mean(), means_b.mean()], y=[0, ymax_mu],
                         mode='lines', line_color=col, line_dash='dash', 
                         name='B Sample Mean'))
fig.update_layout(title='Mu Estimates',
                  xaxis_title='mu',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 2])
fig.show()



fig = go.Figure()
col = 'red'
fig.add_trace(go.Scatter(x=x_sg, y=sigma_dist_a, 
                         mode='lines', line_color=col, 
                         name=f'A Sigma Dist'))
fig.add_trace(go.Scatter(x=[means_a.std(), means_a.std()], y=[0, ymax_sg],
                         mode='lines', line_color=col, line_dash='dash', 
                         name='A Sample Std'))
col = 'blue'
fig.add_trace(go.Scatter(x=x_sg, y=sigma_dist_b,
                         mode='lines', line_color=col, 
                         name=f'B Sigma Dist'))
fig.add_trace(go.Scatter(x=[means_b.std(), means_b.std()], y=[0, ymax_sg],
                         mode='lines', line_color=col, line_dash='dash', 
                         name='B Sample Std'))
fig.update_layout(title='Sigma Estimates',
                  xaxis_title='Sigma',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 2])
fig.show()



fig = go.Figure()
col = 'red'
fig.add_trace(go.Scatter(x=x_post, y=exact_dist_a.pdf(x_post), 
                         mode='lines', line_dash='dash', line_color=col, opacity=0.3,
                         name='A Exact'))
fig.add_trace(go.Scatter(x=x_post, y=sample_clt_dist(means_a).pdf(x_post), 
                         mode='lines', line_color=col, opacity=0.5,
                         name=f"A Sample CLT"))
fig.add_trace(go.Histogram(x=post_means_a, histnorm='probability density',
                           opacity=0.3, marker_color=col,
                           name='A Posterior Means'))
col = 'blue'
fig.add_trace(go.Scatter(x=x_post, y=exact_dist_b.pdf(x_post), 
                         mode='lines', line_dash='dash', line_color=col, opacity=0.3,
                         name='B Exact'))
fig.add_trace(go.Scatter(x=x_post, y=sample_clt_dist(means_b).pdf(x_post), 
                         mode='lines', line_color=col, opacity=0.5,
                         name=f"B Sample CLT"))
fig.add_trace(go.Histogram(x=post_means_b, histnorm='probability density', 
                           opacity=0.3, marker_color=col,
                           name='B Posterior Means'))
fig.update_layout(title='Posterior Sample Means',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.update_layout(xaxis_range=[0, 5])
fig.show()

Распределения апостериорных средних существенно пересекаются, чего не происходит с распределениями $\mu$ и $\sigma$. 

Для выбора одной из групп нужно сравнить распределения. Для этого посмотреть отношение и также оценить вероятности того, что значение параметров в одной из групп больше другой.

При этом возникает вопрос, на какие распределения ориентироваться при выборе варианта - распределения $\mu$ или апостериорных средних? Также, то делать с $\sigma$? Можно ожидать, что по мере набора данных распределение $\mu_i$ будет сужаться с центром на реальном среднем. Распределение апострериорных средних характеризует средние значения в сэмпле из n_split точек. Это не то же самое, что истинное среднее - у этой величины больше дисперсия (поэтому распределения апостериорных средних на графиках выше перекрываются). При сравнении групп нужно выбрать группу именно с большим средним, поэтому нужно ориентироваться на распределение $\mu$. Т.к. распределение $\mu$ проинтегрировано по $\sigma$, то оно не зависит от $\sigma$.  

<Еще возникает вопрос с зависимостью оценки от n_split>.

Для оценки вероятностей того, что значения в одной группе больше другой можно сгенерировать и сравнить сэмплы соответствующих распределений.

In [ ]:
def p_mu_b_ge_mu_a(pars_a, pars_b, mu_size=30000):
    mu_dist_a = mu_marginal_distrib(pars_a)
    mu_dist_b = mu_marginal_distrib(pars_b)
    return np.sum(mu_dist_b.rvs(size=mu_size) >= mu_dist_a.rvs(size=mu_size)) / mu_size

def p_sigma_b_ge_sigma_a(pars_a, pars_b, sg_size=30000):
    lambdas_a = lambda_marginal_distrib(pars_a).rvs(size=sg_size)
    lambdas_b = lambda_marginal_distrib(pars_b).rvs(size=sg_size)
    sigmas_a = pars_a.sigma / np.sqrt(lambdas_a)
    sigmas_b = pars_b.sigma / np.sqrt(lambdas_b)
    return np.sum(sigmas_b >= sigmas_a) / sg_size

def p_expected_b_ge_expected_a(pars_a, pars_b, post_size=30000):
    post_means_a = post_means_rvs(pars_a, size=post_size)
    post_means_b = post_means_rvs(pars_b, size=post_size)
    return np.sum(post_means_b >= post_means_a) / post_size

In [ ]:
print(f'P(mu_B > mu_A): {p_mu_b_ge_mu_a(pars_a[-1], pars_b[-1])}')
print(f'P(sigma_B > sigma_A): {p_sigma_b_ge_sigma_a(pars_a[-1], pars_b[-1])}')
print(f'P(E[B] > E[A]): {p_expected_b_ge_expected_a(pars_a[-1], pars_b[-1])}')

Также для сравнения может быть удобно построить отношение распределений.

In [ ]:
#mu rel
mu_dist_a = mu_marginal_distrib(pars_a[-1]).rvs(size=10000)
mu_dist_b = mu_marginal_distrib(pars_b[-1]).rvs(size=10000)
mu_rel = mu_dist_b / mu_dist_a

fig = go.Figure()
fig.add_trace(go.Histogram(x=mu_rel, histnorm='probability density',
                           name='Mu Relation'))
fig.add_vline(x=1, line_dash='dash')
fig.update_layout(title='Dist',
                  xaxis_title='mu_B / mu_A',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.update_layout(xaxis_range=[0, 2])
fig.show()

#post means rel
post_means_a = post_means_rvs(pars_a[-1], size=30000)
post_means_b = post_means_rvs(pars_b[-1], size=30000)
post_means_rel = post_means_b / post_means_a

fig = go.Figure()
fig.add_trace(go.Histogram(x=post_means_rel, histnorm='probability density',
                           name='Posterior Means Relation'))
fig.add_vline(x=1, line_dash='dash')
fig.update_layout(title='Dist',
                  xaxis_title='Posterior B/Posterior A',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.update_layout(xaxis_range=[0, 5])
fig.show()


print(f"E[mu_B/mu_A] = {mu_rel.mean()}")
print(f"Posterior E[E[B]/E[A]] = {post_means_rel.mean()}")

print(f"Exact E[B]/E[A] = {exp_info['exact_mean']['B'] / exp_info['exact_mean']['A']}")
print(f"Sample E[B]/E[A] = {exp_info['sample_mean']['B'] / exp_info['sample_mean']['A']}")

### Зависимость 95HPDI $\mu$

Для оценки зависимости 95HPDI $\mu$ от количества точек нужно смотреть зависимость распределения от $i$:

$$
P(\mu) = N(\mu; \mu_i, \frac{\sigma_0^2}{k_i \lambda} ) Gamma(\lambda; a_i, b_i) .
$$

Интересует дисперсия $\frac{\sigma_0^2}{k_i \lambda}$.
В ней с ростом числа наблюдений $i$:
$$
k_{i+1} = k_i + 1 .
$$

Также гамма-распределение приближается нормальным распределением:
$$
Gamma(\lambda; a_i, b_i) \to Normal().
$$

Аналитически считать сложно.  
Можно посмотреть численную зависимость $P(\mu_B > \mu_A)$ от числа экспериментальных точек.

Построить график $\frac{\sigma_0^2}{k_i \lambda}$ с ростом $i$.  
Сделать замену переменных (проверить):

$$
P(\sigma | data) = Gamma\left( \frac{\sigma_0^2}{k_i \sigma^2 }; a_i, b_i \right) \frac{2 \sigma_0^2}{k_i \sigma^3}, 
\quad \sigma > 0 .
$$

Зафиксировать отношение $\frac{a_i}{b_i}$. Увеличивать $i$. Строить среднее $\sigma_{\mu}$ от $i$:

## Зависимость вероятности $P(\mu_B > \mu_A)$ от количества точек и продолжение эксперимента до достижения определенного уровня уверенности

Изменение вероятности $P(\mu_B > \mu_A)$ по мере набора данных приведено ниже. При расчете количества точек учитывается, что каждое обновление параметров требует $n_{split}$ точек. 

In [ ]:
pb_gt_pa = [p_mu_b_ge_mu_a(a, b) for a, b in zip(pars_a, pars_b)]
n_points = [i * n_split for i in range(0, len(pars_a))]

fig = go.Figure()
fig.add_trace(go.Scatter(x=n_points, y=pb_gt_pa,
                         name='P(mu_b >= mu_a)'))
fig.update_layout(title='Dist',
                  xaxis_title='N Points',
                  yaxis_title='P(mu_b >= mu_a)',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.update_layout(xaxis_range=[0, 1000], yaxis_range=[0,1])
fig.add_hline(y=0.05, line_dash='dash')
fig.add_hline(y=0.95, line_dash='dash')
fig.show()

Фиксируются $\mu$ и $\sigma$.  
Считается, сколько нужно ждать, пока уверенность не дойдет до определенного уровня.  
При этом можно увеличивать n_split и снижать дисперсию средних.  
Но это пока игнорируется.

In [ ]:
n_simulations = 10

post_pars_a = post_params_rvs(pars_a[-1], n_simulations)
post_pars_b = post_params_rvs(pars_b[-1], n_simulations)

#print(post_pars_a)
#print(post_pars_b)

step = 5000
nmax = 100000

for mu, l, s in post_pars_a:
    post_means_a = stats.norm.rvs(loc=mu, scale=sigma / np.sqrt(l), size = nmax)

#сэмплить параметры mu,l, s
#сгенерировать распределение средних
#с шагом step 
#  добавить сгенерированные точки к имеющимся данным; 
#  оценить параметры mu, l
#  посчитать вероятность mu_a > mu_b ? или P(E[B] > E[A]) - для этого придется еще раз сэмплить апостериорное распределение и 

Не забыть домножить количество наблюдений средних на n_split, чтобы перейти к числу наблюдений.

# Приложение: сопряженное априорное распределение к нормальному распределению

$$
N(x ; \lambda, \mu, \sigma_0^2) = \frac{\lambda^{1/2}}{\sqrt{2 \pi \sigma_0^2}} e^{- \frac{\lambda}{2 \sigma_0^2} (x-\mu)^2}
$$

$$
P(model \mid data) \propto P(data | model) P(model)
$$

$$
P(data | model) = N(x ; \lambda, \mu, \sigma_0^2) = \frac{\lambda^{1/2}}{\sqrt{2 \pi \sigma_0^2}} e^{- \frac{\lambda}{2 \sigma_0^2}(x - \mu)^2}
$$

Можно показать, что сопряженным априорным распределением будет

$$
P(model) = N(\mu ; \lambda, \mu_0, \frac{\sigma_0^2}{k_0}) Gamma(\lambda; a, b)
\\
Gamma(\lambda; a,b) = \frac{b^a}{\Gamma(a)} \lambda^{a-1} e^{-b \lambda}, \quad x>0, \quad a,b>0 
$$

$$
P(model \mid data) = P(\mu, \lambda | x ) \propto 
\frac{\lambda^{1/2}}{\sqrt{2 \pi \sigma_0^2}} e^{- \frac{\lambda}{2 \sigma_0^2}(x - \mu)^2}
\frac{(k_0 \lambda)^{1/2}}{\sqrt{2 \pi \sigma_0^2}} e^{- \frac{k_0 \lambda}{2 \sigma_0^2}(\mu - \mu_0)^2}
\frac{b^a}{\Gamma(a)} e^{-b \lambda} {\lambda}^{a-1}
$$

Выражение в показателе экспонент можно записать в виде

$$
\begin{align}
\frac{\lambda}{2 \sigma_0^2}(x - \mu)^2 + 
\frac{k_0 \lambda}{2 \sigma_0^2}(\mu - \mu_0)^2 +
\lambda b
& =
\frac{\lambda}{2 \sigma_0^2}
\left( \mu^2(k_0+1) - 2\mu(x + k_0 \mu_0) \right) + 
\frac{\lambda}{2 \sigma_0^2}(x^2 + k_0 \mu_0^2 + 2 b \sigma_0^2)
\\
& =
\frac{\lambda (k_0 + 1)}{2 \sigma_0^2}
\left( \mu - \frac{x + k_0 \mu_0}{k_0 + 1} \right)^2
+
\frac{\lambda}{2 \sigma_0^2} \left( x^2 + k_0 \mu_0^2 + 2 b \sigma_0^2 - \frac{(x + k_0 \mu_0)^2}{k_0 + 1} \right)
\\
& =
\frac{\lambda (k_0 + 1)}{2 \sigma_0^2}
\left( \mu - \frac{x + k_0 \mu_0}{k_0 + 1} \right)^2
+
\frac{\lambda}{2 \sigma_0^2} \left( \frac{k_0}{k_0+1} (x - \mu_0)^2 + 2 b \sigma_0^2 \right)
\end{align}
$$

После подстановки, факторы, зависящие только от $\mu$ и $\lambda$:

$$
P(\mu, \lambda | x ) 
\propto 
\lambda^{1/2} e^{- \frac{\lambda (k_0 + 1)}{2 \sigma_0^2} \left( \mu - \frac{x + k_0 \mu_0}{k+1} \right)^2}
{\lambda}^{a-1/2} 
e^{- \frac{\lambda}{2 \sigma_0^2} \left( \frac{k_0}{k_0 + 1} (x - \mu_0)^2 + 2 b \sigma_0^2 \right) }
$$


Апостериорное распределение имеет вид

$$
P(\mu, \lambda | x ) = N(\mu; \lambda, \mu_p, \frac{\sigma_0^2}{k_p} ) Gamma(\lambda; a_p, b_p)
\\
k_p = k_0 + 1
\\
\mu_p = \frac{x + k_0 \mu_0}{k_0 + 1}
\\
a_p = a + 1/2
\\
b_p = b + \frac{k_0}{k_0 + 1} \frac{(x - \mu_0)^2}{2 \sigma_0^2}
$$

In [ ]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html
#scale=1/beta
#центр gamma должен быть на историческом значении дисперсии
#mean = a/b

x = np.linspace(0, 10, 10000)
fig = go.Figure()

for a,b in [(1,1), (1,3), (1, 1/3), (2, 1), (2, 2), (0.9, 1)]:
    fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=a, scale=1/b), mode='lines', name=f'a={a}, b={b}'))

fig.update_layout(title='Gamma Dist',
                  xaxis_title='$\lambda$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)

fig.show()

**Интерпретация:**  

Есть сэмпл. Есть препдоложение, что он получен из нормального распределения.  
Можно оценить параметры исходного распределения 
$\mu, \sigma$ по значениям сэмпла $\mu=\bar{x}, \sigma=\sigma_x$.  
Но уверенность этой оценки слишком завышена.    
Нужно также посмотерть окрестные параметры, например 
$\mu \in [\bar{x}-3 \sigma_x, \bar{x} + 3 \sigma_x]$, $\sigma \in [1/3 \sigma_x, 3 \sigma_x]$.  

Центр поисков среднего - точка $\mu_0$.  
Средние $\mu$ распределены вокруг нее нормально.  
Дисперсия - выбирается естесственный масштаб $\sigma_0$.  
$k_0$ - масштабирующий $\sigma_0$ .   
$p(\mu) = N(\mu; \mu_0, \frac{\sigma_0^2}{k_0})$  
Из соображений сопряженности приходится добавить параметр $\lambda$:  
$p(\mu) = N(\mu; \mu_0, \frac{\sigma_0^2}{k_0 \lambda})$ 

Можно выбрать $k_0 = 1/9$. 
Т.е. исходно дисперсия $(\mu - \mu_0) \sim 3 \sigma_0$

$\lambda$ - масштабирующий множитель для $\sigma_0^2$.  
Вместо того, чтобы задавать распределение $\sigma^2$, можно менять масштабирующий множитель $\lambda$.  
Если $\sigma_0$ близко к $\sigma$, то $\lambda$ должен быть порядка $1$.   
Из соображений сопряженности распределение $\lambda$ будет иметь вид гамма-распределения

$p(\lambda) = \Gamma(\lambda; a, b)$

Мода гамма-распределения $\lambda_{max(p)} = \frac{a-1}{b}$ при $a \ge 1$.  
Исходными параметрами можно выбрать a=2, b=1.  
График выглядит примерно так, как нужно.  
Тем более, что мода с такими параметрами будет равна 1.  


При обновлении данных:  
Будет уточнено значение $\mu_a$.  
Причем чем по большему количеству данных было построено $\mu_a$, тем больший вес оно будет иметь.  
$$
\mu_p = \frac{x + k_0 \mu_0}{k_0 + 1}
$$

Дисперсия $\mu$ вокруг $\mu_0$ будет сужаться по мере набора данных.  
Это происходит за счет увеличения масштабирующего множителя $k$.  

$$
k_p = k_0 + 1
$$

Распределение $\lambda$ будет сужаться (будет обновляться $a$ и $b$).   
Мода будет $\lambda_{max(p)} = \frac{a-1}{b}$.  
Т.е. распределение будет "сужаться" вокруг этого отношения.  
Если $\mu_0$ и $\sigma_0^2$ были близки к реальным, то $a$ и $b$ будут увеличиваться примерно на 1/2 с каждой новой точкой.  
Если нет, то будут сходиться к правильному отношению.  

$$
a_p = a + 1/2,
\qquad
b_p = b + \frac{k_0}{k_0 + 1} \frac{(x - \mu_0)^2}{2 \sigma_0^2}
$$

In [ ]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html
#scale=1/beta
#центр gamma должен быть на историческом значении дисперсии
#mean = a/b

x = np.linspace(0, 10, 10000)
fig = go.Figure()

for a,b in [(2,1), (2.5,1), (3, 1), (11, 5), (11, 10), (11, 9.7), (11, 11.3)]:
    fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=a, scale=1/b), mode='lines', name=f'a={a}, b={b}'))
#fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=1, scale=1/b), mode='lines', name=f'a={a}, b={b}'))
#fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=5, scale=1/b), mode='lines', name=f'a={a}, b={b}'))
#fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=2, scale=1/b), mode='lines', name=f'a={a}, b={b}'))

fig.update_layout(title='Gamma Dist',
                  xaxis_title='$\lambda$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)

fig.show()

Маржинальное распределение $P(\lambda | x)$ по конструкции будет гамма-распределением:

$$
P(\lambda | x) = Gamma(\lambda; a, b)
\\
Gamma(\lambda; a,b) = \frac{b^a}{\Gamma(a)} \lambda^{a-1} e^{-b \lambda}, \quad x>0, \quad a,b>0 
$$

Можно показать, что маржинальное распределение $P(\mu | x)$ будет t-распределением:

$$
\begin{align}
P(\mu | x) 
= \int_0^\infty d\lambda P(\mu, \lambda) d\lambda 
& = 
\int_0^\infty d\lambda 
\frac{(k \lambda)^{1/2}}{\sqrt{2 \pi \sigma_0^2}} e^{- \frac{k_0 \lambda}{2 \sigma_0^2}(\mu - \mu_0)^2}
\frac{b^a}{\Gamma(a)} e^{-b \lambda} {\lambda}^{a-1}
\\
& \propto
\int_0^\infty d\lambda 
\lambda^{a + 1/2 - 1} e^{- \lambda \left( \frac{k_0}{2 \sigma_0^2}(\mu - \mu_0)^2 + b \right) }
\end{align}
$$
Выражение под интегралом по форме совпадает с гамма-распределением.  
Т.к. $\int_0^\infty d\lambda Gamma(\lambda; \alpha, \beta) = 1$, 
то $\int_0^\infty d\lambda \lambda^{\alpha-1} e^{-\beta \lambda} = \Gamma(\alpha) {\beta}^{-\alpha}$

После подстановки
$$
\begin{align}
P(\mu | x) 
& \propto_{\mu} 
\left(b + \frac{k_0}{2 \sigma_0^2}(\mu - \mu_0)^2 \right)^{-(a+1/2)}
\\
& \propto_{\mu} 
\left(1 + \frac{k_0}{2 \sigma_0^2 b}(\mu - \mu_0)^2 \right)^{-(a+1/2)}
\end{align}
$$

Обобщенное t-распределение имеет вид  
https://en.wikipedia.org/wiki/Student's_t-distribution#Generalized_Student's_t-distribution  
$$
t(x | \nu, \mu_t, \sigma_t^2) 
\propto_x 
\left(1+\frac{1}{\nu} \frac{ (x - \mu_t)^2 }{\sigma_t^2 } \right)^{-(\nu+1)/2}
$$

Поэтому при 
$$
a = \frac{\nu}{2}, \quad \nu \sigma_t^2 = \frac{2 \sigma_0^2 b}{k_0}
$$

Маржинальное распределение $P(\mu | x)$:
$$
P(\mu | x) = t(\mu | \nu = 2a, \mu_t = \mu_0, \sigma_t^2 = \frac{\sigma_0^2}{k_0} \frac{b}{a} )
$$

$\sigma_t^2 = \frac{\sigma_0^2}{k_0} \frac{b}{a}$  
$\frac{\sigma_0^2}{k_0}$ - дисперсия средих вокруг $\mu_0$.  
$\frac{b}{a}$ - характеризует отношение реальной дисперсии к $\sigma_0$.  
$p(\lambda)$ будет локализовыватсья вокруг $(a-1)/b$.

По мере добавления данных:  
Увеличивается число степеней свободы $\nu$, что ведет к сужению распределения.  
Увеличивается $k_0$, что ведет к снижению дисперсии.  
$\frac{b}{a}$ стабилизируется на каком-то значении (порядка 1, если удачно выбраны $\mu_0, \sigma_0^2$)

In [ ]:
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html#scipy.stats.t

x = np.linspace(0, 10, 10000)
fig = go.Figure()

for df, scale, loc in [(2, 1, 5), (20, 1, 5), (2, 0.7, 5), (20, 0.7, 5)]:
    fig.add_trace(go.Scatter(x=x, y=stats.t.pdf(x, df=df, loc=loc, scale=scale), mode='lines', name=f'df={df}, loc={loc}, scale={scale}'))

fig.update_layout(title='t-Dist',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)

fig.show()

# Приложение: оценка параметров нормального распределения по сэмплу

Пусть есть нормальное распределение.  
Параметры $\mu, \sigma$.
Есть сэмпл $n$ точек.
Нужно по сэмплу угадать параметры.

Есть ожидание, что распределение для $\mu$ должно будет совпасть с распределением средних.  
Т.е. нормальное с параметрами ($\mu_s$, $\sigma_s^2 / N$).  
Единственное, поиск начинается с параметров $\mu_0$, $\sigma_0$, поэтому не совсем совпадет.

In [ ]:
mu_exact = 20
s_exact = 3
sample_size = 100

print(stats.norm.std(loc=mu_exact, scale=s_exact))
samp = stats.norm.rvs(loc=mu_exact, scale=s_exact, size=sample_size)

x = np.linspace(0, 50, 10000)
fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=mu_exact, scale=s_exact), mode='lines', name='Exact'))
fig.add_trace(go.Histogram(x=samp, histnorm='probability density', name='Sample'))
fig.update_layout(title='Dist',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
#todo: add distrib mean and sample mean
fig.show()

print(f'exact mu_0 = {mu_exact}, std={s_exact}')
print(f'sample avg = {np.mean(samp)}, std={np.std(samp)}')

In [ ]:
def update_parameters(x, mu, sigma, k, a, b):
    #if your function has 10 parameters ...
    mu_p = (x + k * mu) / (k + 1)
    sigma_p = sigma    
    k_p = k + 1
    a_p = a + 1/2
    b_p = b + k / (k + 1) * (x - mu)**2 / (2 * sigma**2)
    return mu_p, sigma_p, k_p, a_p, b_p

In [ ]:
#mu_0 = np.mean(samp)
#sigma_0 = np.std(samp)
mu_0 = 1
sigma_0 = 1
k_0 = 1 / 25
a_0 = 2
b_0 = 1

print((mu_0, sigma_0, k_0, a_0, b_0))

In [ ]:
pars = []
pars.append((mu_0, sigma_0, k_0, a_0, b_0))

for x in samp:
    new_pars = update_parameters(x, *pars[-1])
    pars.append(new_pars)
    
print(len(pars))
display(pars[0], pars[1], pars[len(pars) // 3], pars[len(pars) // 2], pars[-1])

In [ ]:
x = np.linspace(0, 50, 30000)

fig = go.Figure()

y_plot_max = 0
for mu, sigma, k, a, b in [pars[0], pars[len(pars) // 3], pars[len(pars) // 2], pars[-1]]:
    df = 2 * a
    loc = mu
    scale = np.sqrt(sigma**2 / k * b / a)
    y_plot = stats.t.pdf(x, df=df, loc=loc, scale=scale)
    y_plot_max = max(max(y_plot), y_plot_max)
    fig.add_trace(go.Scatter(x=x, y=y_plot, mode='lines', name=f'df={df}, loc={loc}, scale={scale}'))

fig.add_trace(go.Scatter(x=[mu_exact, mu_exact], 
                         y=[0, y_plot_max], 
                         line_width=0.8, mode='lines', line_color='black', name='Exact Mean'))
fig.add_trace(go.Scatter(x=[np.mean(samp), np.mean(samp)],
                         y=[0, y_plot_max],
                         line_width=0.8, mode='lines', line_color='black', line_dash="dash", name='Sample Mean'))

fig.add_trace(go.Scatter(x=x, 
                         y=stats.norm.pdf(x=x, loc=np.mean(samp), scale=np.std(samp) / np.sqrt(len(samp))), 
                         mode='lines', name=f'Normal, mean=sample_mean, std=stderrmean'))

fig.update_layout(title='Mean Distribution',
                  xaxis_title='$\mu$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


fig = go.Figure()

for mu, sigma, k, a, b in [pars[0], pars[len(pars) // 3], pars[len(pars) // 2], pars[-1]]:
    fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=a, scale=1/b), mode='lines', name=f'a={a}, b={b}'))

#http://www.math.wm.edu/~leemis/chart/UDR/PDFs/GammaNormal1.pdf
stderr_mean = a / b
stderr_sigma = np.sqrt(a) / b
fig.add_trace(go.Scatter(x=x, 
                         y=stats.norm.pdf(x=x, loc=stderr_mean, scale=stderr_sigma), 
                         mode='lines', name=f'Normal, mean=stderr_mean, std=stderr_sigma'))
    
fig.update_layout(title='Dispersion Scaling Distribution',
                  xaxis_title='$\lambda$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 10])
fig.show()

In [ ]:
x = np.linspace(0.01, 50, 30000)

fig = go.Figure()
y_plot_max = 0
for mu, sigma, k, a, b in [pars[0], pars[len(pars) // 3], pars[len(pars) // 2], pars[-1]]:
    x_plot = np.sqrt(sigma * sigma / x)
    y_plot = stats.gamma.pdf(x, a=a, scale=1/b)
    y_plot_max = max(y_plot_max, max(y_plot))
    fig.add_trace(go.Scatter(x=x_plot, y=y_plot, mode='lines', name=f'a={a}, b={b}'))

fig.add_trace(go.Scatter(x=[s_exact, s_exact], 
                         y=[0, y_plot_max], 
                         line_width=0.8, mode='lines', line_color='black', name='Exact Stdev'))
fig.add_trace(go.Scatter(x=[np.std(samp), np.std(samp)],
                         y=[0, y_plot_max],
                         line_width=0.8, mode='lines', line_color='black', line_dash="dash", name='Sample Stdev'))

# Гамма-распределение для lambda стремится к нормальному
# Потом в нормальном распределении можно перейти к другим переменным и получить аналитическое для sigma
stderr_mean = a / b
stderr_sigma = np.sqrt(a) / b
x_plot = np.sqrt(sigma * sigma / x)
y_plot = stats.norm.pdf(x=x, loc=stderr_mean, scale=stderr_sigma)
fig.add_trace(go.Scatter(x=x_plot,
                         y=y_plot, 
                         mode='lines', name=f'Normal, mean=stderr_mean, std=stderr_sigma'))


fig.update_layout(title='Std Distribution',
                  xaxis_title='$\sigma$',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 10])
fig.show()

standard error of the sample standard deviation  
https://stats.stackexchange.com/questions/631/standard-deviation-of-standard-deviation  
https://stats.stackexchange.com/questions/156518/what-is-the-standard-error-of-the-sample-standard-deviation  
http://www.math.wm.edu/~leemis/chart/UDR/PDFs/GammaNormal1.pdf 
http://www.stat.columbia.edu/~gelman/research/published/taumain.pdf
https://en.wikipedia.org/wiki/Scaled_inverse_chi-squared_distribution

https://en.wikipedia.org/wiki/Cochran's_theorem#Estimation_of_variance

## Заключение

## Благодарности

## Ссылки

## Черновики

**Is there an analog of central limit theorem for distribution of dispersion?**

A Central Limit Theorem states that for a distribution with a well-defined mean $\mu$ and dispersion $\sigma^2$
distribution of averages approaches normal distribution 

$$
\frac{x_1 + x_2 + \dots + x_N}{N} \to \mathcal{N}(\mu, \frac{\sigma^2}{N}) 
$$

Is there an analogous relation for dispersion

$$
\frac{(x_1 - \mu)^2 + (x_2 - \mu)^2 + \dots + (x_N - \mu)^2}{N} \to p(\mu, \sigma, N) ?
$$

https://en.wikipedia.org/wiki/Chi-squared_distribution#Asymptotic_properties  
https://en.wikipedia.org/wiki/Cochran's_theorem#Estimation_of_variance  
https://stats.stackexchange.com/questions/429782/why-is-the-limit-of-a-chi-squared-distribution-a-normal-distribution

In [ ]:
a = 1
sample_len = 10
n_samples = 10000
samp = stats.gamma.rvs(a=a, size=(n_samples, sample_len))
stdevs = np.array([x.std() for x in samp])

clt_mu = stats.gamma.mean(a=a)
clt_stdev = stats.gamma.std(a=a) / np.sqrt(n_samples)

x = np.linspace(0, 200, 10000)
fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=a), mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_vline(stats.gamma.std(a=a), name='Original Distribution Std')
fig.add_trace(go.Histogram(x=stdevs, histnorm='probability density', name='Means of Samples'))

#fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=clt_mu, scale=clt_stdev), mode='lines', name='CLT Means Distrib'))
fig.add_trace(go.Scatter(x=x / sample_len, y=stats.chi2.pdf(x, df=sample_len-1) * sample_len, mode='lines', name='CLT Dispersion Distrib'))

fig.update_layout(title='Dist',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

**Разные ссылки**

https://en.wikipedia.org/wiki/Conjugate_prior  
https://www.cs.ubc.ca/~murphyk/Papers/bayesGauss.pdf  
https://deebuls.github.io/devblog/probability/python/plotting/matplotlib/2020/05/19/probability-normalinversegamma.html     
https://people.eecs.berkeley.edu/~jordan/courses/260-spring10/lectures/lecture5.pdf  
https://en.wikipedia.org/wiki/Normal_distribution#With_unknown_mean_and_unknown_variance  
https://statproofbook.github.io/P/ugkv-prior    
http://users.isr.ist.utl.pt/~wurmd/Livros/school/Bishop%20-%20Pattern%20Recognition%20And%20Machine%20Learning%20-%20Springer%20%202006.pdf (sec 2.3.6, 2.3.7)

**Формулировки ЦПТ**

$$
\sqrt{n} \frac{\bar{X}_n - \mu}{\sigma} \to N(0, 1)
$$

$$
\sqrt{n}(\bar{X}_n - \mu) \to N(0, \sigma^2)
$$

$$
\frac{S_n - \mu n}{\sigma \sqrt{n}} \to N(0, 1), \quad n \to \infty.
$$

$$
\frac{\sum_{i=1}^n X_i - \mu n}{\sigma \sqrt{n}} \to N(0, 1), \quad n \to \infty.
$$

$$
\frac{X_1 + X_2 + \dots + X_n}{n} \to N(\mu, \sigma^2/n), \quad n \to \infty.
$$

$$
\frac{(X_1 + X_2 + \dots + X_n) - \mu n}{\sigma \sqrt{n}} \to N(0, 1), \quad n \to \infty.
$$

**Независимые наблюдения в ЦПТ**

*Еще одна возможная проблема - считать элементы сэмпла независимыми, хотя на самом деле они не независимы (нарушается i.i.d.).   
Пример - считать независимыми последовательные сессии одного и того же пользователя.*  

Посмотреть пример:  
500 бросков монетки.  
В одном случае независимые. Т.е. Binom(n=500, p=0.5).   
В другом при выпадении 1 сразу добавляется еще 1.   
Распределения будут разные.  
Сходу не ясно, как будет работать ЦПТ?  

**Гамма или обратное гамма-распределение вместо нормального для выбросов**

In [ ]:
c = 1.7
sample_len = 500
n_samples = 30000

exact_dist = stats.lomax(c=c)
samp = exact_dist.rvs(size=(n_samples, sample_len))
means = np.array([x.mean() for x in samp])
clt_mu = exact_dist.mean()
clt_stdev = exact_dist.std() / np.sqrt(sample_len)
means_stdev = means.std() 
means_stdev = means_stdev / 2

gamma_alpha = means.mean()**2 / means.std()**2 
gamma_alpha = gamma_alpha
gamma_beta = means.mean() / means.std()**2 
gamma_beta = gamma_beta * 2.3
gamma_scale = 1 / gamma_beta
gamma_loc = 0.8

invgamma_alpha = (means.mean()**2 / means.std()**2 + 2)
invgamma_beta = means.mean() * (means.mean()**2 / means.std()**2 + 1)
invgamma_beta = invgamma_beta / 2
#invgamma_scale = 1 / invgamma_beta


x = np.linspace(0.001, 30, 10000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), mode='lines', name='Exact Dist'))
fig.add_trace(go.Histogram(x=samp[0], histnorm='probability density', name='Sample'))
fig.update_layout(title=f'Original Distribution and a Sample of Length {sample_len}',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.show()


fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=exact_dist.pdf(x), 
                         mode='lines', line_dash='dash', name='Original Distribution'))
fig.add_vline(exact_dist.mean(), name='Original Distribution Mean')
fig.add_trace(go.Histogram(x=means, histnorm='probability density', name='Means of Samples'))
fig.add_trace(go.Scatter(x=x, y=stats.norm.pdf(x, loc=clt_mu, scale=means_stdev), 
                         mode='lines', name='CLT-like Normal'))
fig.add_trace(go.Scatter(x=x, y=stats.gamma.pdf(x, a=gamma_alpha, scale=gamma_scale, loc=gamma_loc), 
                         mode='lines', name='Gamma'))
fig.add_trace(go.Scatter(x=x, y=stats.invgamma.pdf(x, a=invgamma_alpha, loc=0.7, scale=invgamma_beta), 
                         mode='lines', name='InvGamma'))
fig.update_layout(title='Sample Means Distribution',
                  xaxis_title='x',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

Обратное гамма-распределение вроде более-менее попадает.

Не ясно, есть ли теоретическое обоснование.  
Есть обобщенная центральная предельная теорема  
https://en.wikipedia.org/wiki/Stable_distribution#A_generalized_central_limit_theorem  
Частный случай "стабильных" распределений - распределение Леви  
https://en.wikipedia.org/wiki/L%C3%A9vy_distribution  
Частный случай распределения Леви - обратное гамма-распределение  
https://en.wikipedia.org/wiki/Inverse-gamma_distribution

Совпадение моды распределения $\sigma$ со значением дисперсии в выборке
сильно зависит от количества наблюдений и от начального значения $\sigma_0$.    
При $\sigma_0=1$ не совпадает, при $\sigma_0=0.1$ совпадает.  
Возможно, нужно использовать часть данных для задания начальных значений параметров.  

**зависимость оценки от n_split**

In [ ]:
# def n_split_initial_parameters(mu, sigma):
#     #todo: use remaining sample points
#     return ConjugateNormalParams(mu=mu, sigma=sigma, k=1/25, a=2, b=1)

# def n_split_compute_posterior_parameters(means):
#     pars = []
#     pars.append(n_split_initial_parameters()) 
#     for x in means:
#         new_pars = update_conj_parameters(x, pars[-1])
#         pars.append(new_pars)
#     return pars

# def compute_posterior_parameters(sample, n_split):
#     means = reshape_and_compute_means(sample, n_split)
#     mean_1 = means[0]
#     sigma_1 = sample[0:n_split].std() / np.sqrt(n_split)
#     pars = []
#     pars.append(initial_parameters(mu=mean_1, sigma=sigma_1)) 
#     for x in means[1:]:
#         new_pars = update_conj_parameters(x, pars[-1])
#         pars.append(new_pars)
#     return pars


fig = go.Figure()
x = np.linspace(0, 10, 10000)

for n_split in [9, 16, 25, 49, 100, 225, 400]:
    print('n_split: ', n_split)
    n_split_means_a = reshape_and_compute_means(samp_a, n_split)
    n_split_means_b = reshape_and_compute_means(samp_b, n_split)
    #n_split_pars_a = n_split_compute_posterior_parameters(n_split_means_a)
    #n_split_pars_b = n_split_compute_posterior_parameters(n_split_means_b)
    n_split_pars_a = compute_posterior_parameters(samp_a, n_split)
    n_split_pars_b = compute_posterior_parameters(samp_b, n_split)
    n_split_mu_dist_a = mu_marginal_distrib(n_split_pars_a[-1])
    n_split_mu_dist_b = mu_marginal_distrib(n_split_pars_b[-1])
    n_split_post_means_a = post_means_rvs(n_split_pars_a[-1], size=30000)
    n_split_post_means_b = post_means_rvs(n_split_pars_b[-1], size=30000)
    
    col = 'red'
    fig.add_trace(go.Scatter(x=x, y=sample_clt_dist(n_split_means_a).pdf(x), 
                         mode='lines', line_color=col, opacity=0.5,
                         name=f"A Sample CLT, {n_split}"))
    col = 'blue'
    fig.add_trace(go.Scatter(x=x, y=sample_clt_dist(n_split_means_b).pdf(x), 
                         mode='lines', line_color=col, opacity=0.5,
                         name=f"B Sample CLT, {n_split}"))
    
    
    n_split_p_mean_b_gt_mean_a = np.sum(n_split_post_means_b > n_split_post_means_a) / len(n_split_post_means_b)
    print(f'P(E[B] > E[A]): {n_split_p_mean_b_gt_mean_a}')

    n_split_mu_size = 10000
    n_split_p_mu_b_gt_mu_a = np.sum(n_split_mu_dist_b.rvs(size=n_split_mu_size) > n_split_mu_dist_a.rvs(size=n_split_mu_size)) / n_split_mu_size
    print(f'P(mu_B > mu_A): {n_split_p_mu_b_gt_mu_a}')


col = 'red'
fig.add_trace(go.Histogram(x=n_split_post_means_a, histnorm='probability density',
                           opacity=0.3, marker_color=col,
                           name='A Posterior Means'))
fig.add_trace(go.Histogram(x=n_split_means_a, histnorm='probability density',
                           opacity=0.5, marker_color=col,
                           name='B Sample Means'))
col = 'blue'
fig.add_trace(go.Histogram(x=n_split_post_means_b, histnorm='probability density', 
                           opacity=0.3, marker_color=col,
                           name='B Posterior Means'))
fig.add_trace(go.Histogram(x=n_split_means_b, histnorm='probability density',
                           opacity=0.5, marker_color=col,
                           name='B Sample Means'))
fig.update_layout(title='Dist',
                  xaxis_title='p',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550,
                  barmode="overlay")
fig.update_layout(xaxis_range=[0, 5])
fig.show()

In [ ]:
fig = go.Figure()
x_sg = np.linspace(0.01, 10, 10000)

for n_split in [9, 16, 25, 49, 100, 225, 400]:
    print('n_split: ', n_split)
    n_split_means_a = reshape_and_compute_means(samp_a, n_split)
    n_split_means_b = reshape_and_compute_means(samp_b, n_split)
    #n_split_pars_a = n_split_compute_posterior_parameters(n_split_means_a)
    #n_split_pars_b = n_split_compute_posterior_parameters(n_split_means_b)
    n_split_pars_a = compute_posterior_parameters(samp_a, n_split)
    n_split_pars_b = compute_posterior_parameters(samp_b, n_split)
    n_split_sigma_dist_a = sigma_marginal_distrib_pdf(x_sg, n_split_pars_a[-1])
    n_split_sigma_dist_b = sigma_marginal_distrib_pdf(x_sg, n_split_pars_b[-1])
    n_split_ymax_sg = np.max([n_split_sigma_dist_a, n_split_sigma_dist_b]) #todo: optimize
    col = 'red'
    fig.add_trace(go.Scatter(x=x_sg, y=n_split_sigma_dist_a, 
                             mode='lines', line_color=col, 
                             name=f'A Sigma Dist, {n_split}'))
    fig.add_trace(go.Scatter(x=[n_split_means_a.std(), n_split_means_a.std()], y=[0, n_split_ymax_sg],
                             mode='lines', line_color=col, line_dash='dash', 
                             name=f'A Sample Std, {n_split}'))
    col = 'blue'
    fig.add_trace(go.Scatter(x=x_sg, y=n_split_sigma_dist_b,
                             mode='lines', line_color=col, 
                             name=f'B Sigma Dist, {n_split}'))
    fig.add_trace(go.Scatter(x=[n_split_means_b.std(), n_split_means_b.std()], y=[0, n_split_ymax_sg],
                             mode='lines', line_color=col, line_dash='dash', 
                             name=f'B Sample Std, {n_split}'))

fig.update_layout(title='Sigma Estimates',
                  xaxis_title='Sigma',
                  yaxis_title='Prob Density',
                  hovermode="x",
                  height=550)
fig.update_layout(xaxis_range=[0, 5])
fig.show()

Если выбрать n_split так, что в сэмпле средних окажется "мало" точек, то 
не получается точно подобрать параметры апостериорного распределения.

Также, если мало точек, нельзя убедительно считать, что распределение средних нормальное.  

При этом CLT-распределение сужается.  
А байесовское - нет.  
Но ведь если при малом количестве точек распределение нормальное, то при большем количестве точек оно скорее
всего останется нормальным.  

Кажется, что тут есть зависимость от начальных параметров.  
Точнее - проблемы из-за $\sigma$.  
Если завысить - распределение будет шире, чем могло бы быть.  
Если занизить - будет уже.  

Баланс - сгенерированное апостериорное распределение более-менее воспроизводит распределение сэмплов.  
И, возможно, распределение ЦПТ.  

## Приложение: сравнение среднего чека одной покупки

In [ ]:
a_prices = np.array([0, 3, 10, 30])
a_probs = np.array([80, 12, 5, 3])

b_prices = np.array([0, 5, 15, 50])
b_probs = np.array([85, 10, 3, 2])

xmax = np.max([a_prices, b_prices])
a_mean = np.sum(a_prices * a_probs / 100)
b_mean = np.sum(b_prices * b_probs / 100)

fig = go.Figure()
fig.add_trace(go.Bar(x=a_prices, y=a_probs, marker_color='red', name='A'))
fig.add_vline(a_mean, line_dash='dash')
fig.update_layout(title='A Prices')
fig.update_layout(yaxis_range=[0,100], 
                  xaxis_range=[0, xmax])
fig.show()

fig = go.Figure()
fig.add_trace(go.Bar(x=b_prices, y=b_probs, marker_color='blue', name='B'))
fig.add_vline(b_mean, line_dash='dash')
fig.update_layout(title='B Prices')
fig.update_layout(yaxis_range=[0,100], 
                  xaxis_range=[0, xmax])
fig.show()

print(f'A probs sum: {np.sum(a_probs)}, B probs sum: {np.sum(b_probs)}')
print(f'A prob buy: {100 - a_probs[0]}, B prob buy: {100 - b_probs[0]}')
print(f'B_prob_buy / A_prob_buy: {(100 - b_probs[0]) / (100 - a_probs[0]) :.2f}')
print(f'A mean: {a_mean} B mean: {b_mean}')
print(f'B mean / A mean: {b_mean / a_mean :.2f}')

In [ ]:
n_users = 1000

a_purs = np.random.choice(a_prices, size=n_users, p=a_probs/100)
b_purs = np.random.choice(b_prices, size=n_users, p=b_probs/100)

print(f'A sample mean: {np.mean(a_purs)}')
print(f'B sample mean: {np.mean(b_purs)}')